In [250]:
import requests
import datetime as dt
from twilio.rest import Client
account_sid = 'AC242189f5c43b109876f91fd001fcb011'
auth_token = '619348870a2c1f4ce65f5bb1cc2055ce'
client = Client(account_sid, auth_token)


Constants

In [251]:
STOCK = "TSLA"
COMPANY_NAME = "Tesla Inc"

stock_api_key = 'FDRJR4OUIDSXOH5V'
stock_api_EndPoint = 'https://www.alphavantage.co/query?'
function='TIME_SERIES_DAILY'
symbol=STOCK

stock_parameters = {
    'function':function,
    'symbol':symbol,
    'apikey':stock_api_key,
    'outputsize':'compact'
}

news_api_key = 'd03bf8377bd647ff930fcfccffe8b10b'
news_api_end_point = 'https://newsapi.org/v2/everything?'
q = COMPANY_NAME

arrow = ""

Calling the api and getting the response

In [252]:
response = requests.get(stock_api_EndPoint,params=stock_parameters)
response.raise_for_status()

STEP 1: Use https://www.alphavantage.co
- When STOCK price increase/decreases by 5% between yesterday and the day before yesterday then print("Get News").

In [253]:
today = dt.date.today()
yesterday = str(today - dt.timedelta(days=2))
day_before_yesterday = str(today - dt.timedelta(days=3))

print(yesterday)

2025-04-23


In [254]:
def percentage_change_in_stock_prices(new,old):
    change = ((new-old)/old) *100.0
    global arrow

    if new > old:
         print(f"Increase of {change:.2f}%")
         arrow = "🔺"
         return f"{change:.2f}"
    elif old > new:
         print(f"Decrase of {change:.2f}%")
         arrow = "🔻"
         return f"{change:.2f}"
    else:
        print(f"no change")
        return f"{change:.2f}"

In [255]:
stock_data = response.json()['Time Series (Daily)']
yesterday_stock_price = float(stock_data[yesterday]['4. close'])
day_before_yesterday_stock_price = float(stock_data[day_before_yesterday]['4. close'])

change = float(percentage_change_in_stock_prices(yesterday_stock_price,day_before_yesterday_stock_price))
change
arrow

Increase of 5.37%


'🔺'

STEP 2: Use https://newsapi.org
- Instead of printing ("Get News"), actually get the first 3 news pieces for the COMPANY_NAME. 

In [256]:
news_parameters = {
    'q':COMPANY_NAME,
    'apiKey':news_api_key,
    'from':day_before_yesterday,
    'to':today,
    'pageSize':3,
    'language':'en'
}

In [257]:
response = requests.get(news_api_end_point,params=news_parameters)
response.raise_for_status()
news_data = response.json()['articles']

news_data

[{'source': {'id': None, 'name': 'KTLA Los Angeles'},
  'author': 'Will Conybeare',
  'title': 'L.A. warehouse said to be owned by Kanye West causing concern for neighbors: ‘We don’t really know what’s going on’',
  'description': 'A warehouse in Los Angeles said to be owned by Ye, also known as Kanye West, is drawing ire from nearby residents. Neighbors told KTLA on Wednesday that the ...',
  'url': 'https://ktla.com/news/local-news/l-a-warehouse-said-to-be-owned-by-kanye-west-causing-concern-for-neighbors-we-dont-really-know-whats-going-on/',
  'urlToImage': 'https://media.zenfs.com/en/ktla_articles_362/cf062aeccbf936589ca5c95ae7025087',
  'publishedAt': '2025-04-23T20:20:21Z',
  'content': 'A warehouse in Los Angeles said to be owned by Ye, also known as Kanye West, is drawing ire from nearby residents. \r\nNeighbors told KTLA on Wednesday that the nondescript white building on the corner… [+2224 chars]'},
 {'source': {'id': None, 'name': 'Forbes'},
  'author': 'Adam Sarhan, Contrib

In [258]:
news_data = {
    i:{
        'headline':news_data[i]['title'],
        'brief':news_data[i]['description']
       } 
             for i in range(len(news_data))
             }
news_data

{0: {'headline': 'L.A. warehouse said to be owned by Kanye West causing concern for neighbors: ‘We don’t really know what’s going on’',
  'brief': 'A warehouse in Los Angeles said to be owned by Ye, also known as Kanye West, is drawing ire from nearby residents. Neighbors told KTLA on Wednesday that the ...'},
 1: {'headline': 'Earnings Preview: What To Expect From Tesla',
  'brief': "Here's a closer look at Tesla's earnings and what Wall Street Analysts Expect will happen after TSLA reports earnings."},
 2: {'headline': 'Tesla at Risk of a ‘Throwaway Year’ Without a New Narrative From Earnings',
  'brief': '(Bloomberg) -- Tesla Inc. investors reeling from one of the stock’s worst-ever quarters are once again looking for Elon Musk to inject excitement back into...'}}

STEP 3: Use https://www.twilio.com
- Send a seperate message with the percentage change and each article's title and description to your phone number. 

In [259]:
for article in news_data:
    post = news_data[article]
    print(post['brief'])

A warehouse in Los Angeles said to be owned by Ye, also known as Kanye West, is drawing ire from nearby residents. Neighbors told KTLA on Wednesday that the ...
Here's a closer look at Tesla's earnings and what Wall Street Analysts Expect will happen after TSLA reports earnings.
(Bloomberg) -- Tesla Inc. investors reeling from one of the stock’s worst-ever quarters are once again looking for Elon Musk to inject excitement back into...


In [260]:
if change >=5 or change <= -5:
    for i in news_data:
        print(i)
        article = news_data[i]
        message = client.messages.create(
            body=f"{STOCK}: {arrow}{change:.2f}%\n"
                f"Headline: {article['headline']}\n"
                f"Brief: {article['brief']}",
            from_='+16693057033',
            to='+918879598537'
        )
        print(f"Sent: {message.status}")

0
Sent: queued
1


TwilioRestException: HTTP 429 error: Unable to create record: Account AC242189f5c43b109876f91fd001fcb011 exceeded the 9 daily messages limit

#Optional: Format the SMS message like this: 
"""
TSLA: 🔺2%
Headline: Were Hedge Funds Right About Piling Into Tesla Inc. (TSLA)?. 
Brief: We at Insider Monkey have gone over 821 13F filings that hedge funds and prominent investors are required to file by the SEC The 13F filings show the funds' and investors' portfolio positions as of March 31st, near the height of the coronavirus market crash.
or
"TSLA: 🔻5%
Headline: Were Hedge Funds Right About Piling Into Tesla Inc. (TSLA)?. 
Brief: We at Insider Monkey have gone over 821 13F filings that hedge funds and prominent investors are required to file by the SEC The 13F filings show the funds' and investors' portfolio positions as of March 31st, near the height of the coronavirus market crash.
"""